In [17]:
import numpy as np
import pandas as pd 

from keras.models import Sequential
from keras import layers
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [5]:
filepath_dict = {'yelp': './data/yelp_labelled.txt',
                'amazon' : './data/amazon_cells_labelled.txt',
                'imdb' : './data/imdb_labelled.txt'}
df_list = []
for source,path in filepath_dict.items():
    df = pd.read_csv(path, names = ['sentence', 'label'], sep= '\t')
    df['source'] = source
    df_list.append(df)

df = pd.concat(df_list)
print(df.iloc[0])

sentence    Wow... Loved this place.
label                              1
source                          yelp
Name: 0, dtype: object


In [6]:
df_yelp = df[df['source'] == 'yelp']
sentences = df_yelp['sentence'].values
y = df_yelp['label'].values

sentences_train,sentences_test,y_train,y_test = train_test_split(sentences,y,test_size=0.25,random_state=1000)

print('Training set: {} '.format(len(sentences_train)))
print('Testing set: {} '.format(len(sentences_test)))

Training set: 750 
Testing set: 250 


In [7]:
# so in order to embed the words they first need to be tokenized (pre-processed)
tokenizer = Tokenizer(num_words=5000)
# this fills the dictionry of 5000 words with the words from the 750 yelp reviews
tokenizer.fit_on_texts(sentences_train)

# creates the vector representation of the sentences where each element is the index of that word in the dictionry
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  #reserves 0 index

print("Vocab Size: ", vocab_size)
print(X_train[1])
print(sentences_train[1])

Vocab Size:  1747
[740, 4, 46, 12, 20, 160, 10, 72, 35, 355, 232]
Sorry, I will not be getting food from here anytime soon :(


In [8]:
# again we make all feature vectors the same size here
maxlen = 100

X_train = pad_sequences(X_train,padding='post',maxlen=maxlen)
X_test = pad_sequences(X_test,padding='post',maxlen=maxlen)

print(X_train[1])

[740   4  46  12  20 160  10  72  35 355 232   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [9]:
# we are only going to use the words in GloVe that are actually in out training set
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [15]:
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    
    embedding_matrix = create_embedding_matrix('/Users/elidangerfield/Desktop/glove.6B/glove.6B.{}d.txt'.format(embedding_dim),
                                           tokenizer.word_index, 
                                           embedding_dim)
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [16]:
param_grid = dict(num_filters=[32, 64, 128],
                  kernel_size=[3, 5, 7],
                  vocab_size=[5000], 
                  embedding_dim=[50,100],
                  maxlen=[100])

In [18]:
epochs = 20
embedding_dim = 50
maxlen = 100
output_file = 'data/output.txt'

In [29]:
model = KerasClassifier(build_fn=create_model,
                            epochs=epochs, batch_size=10,
                            verbose=True)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=3, verbose=1, n_iter=5)
grid_result = grid.fit(X_train, y_train)

    # Evaluate testing set
test_accuracy = grid.score(X_test, y_test)


Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
562/562 [==============================] - 4s 7ms/step - loss: 0.6877 - acc: 0.5676
Epoch 2/20
562/562 [==============================] - 2s 4ms/step - loss: 0.6205 - acc: 0.8060
Epoch 3/20
562/562 [==============================] - 2s 3ms/step - loss: 0.4258 - acc: 0.9306
Epoch 4/20
562/562 [==============================] - 2s 3ms/step - loss: 0.1809 - acc: 0.9662
Epoch 5/20
562/562 [==============================] - 2s 4ms/step - loss: 0.0594 - acc: 0.9947
Epoch 6/20
562/562 [==============================] - 2s 3ms/step - loss: 0.0217 - acc: 1.0000
Epoch 7/20
562/562 [==============================] - 2s 4ms/step - loss: 0.0099 - acc: 1.0000
Epoch 8/20
562/562 [==============================] - 2s 3ms/step - loss: 0.0060 - acc: 1.0000
Epoch 9/20
562/562 [======================

563/563 [==============================] - 2s 3ms/step - loss: 9.7854e-04 - acc: 1.0000
Epoch 15/20
563/563 [==============================] - 2s 3ms/step - loss: 8.1933e-04 - acc: 1.0000
Epoch 16/20
563/563 [==============================] - 2s 3ms/step - loss: 6.9720e-04 - acc: 1.0000
Epoch 17/20
563/563 [==============================] - 2s 3ms/step - loss: 6.0085e-04 - acc: 1.0000
Epoch 18/20
563/563 [==============================] - 2s 3ms/step - loss: 5.2051e-04 - acc: 1.0000
Epoch 19/20
563/563 [==============================] - 2s 3ms/step - loss: 4.5599e-04 - acc: 1.0000
Epoch 20/20
187/187 [==============================] - 0s 2ms/step
Epoch 1/20
562/562 [==============================] - 3s 5ms/step - loss: 0.6921 - acc: 0.5231
Epoch 2/20
562/562 [==============================] - 1s 2ms/step - loss: 0.6267 - acc: 0.7776
Epoch 3/20
562/562 [==============================] - 1s 2ms/step - loss: 0.4145 - acc: 0.9093
Epoch 4/20
562/562 [==============================] - 1s 3ms

563/563 [==============================] - 1s 2ms/step - loss: 5.4503e-04 - acc: 1.0000
Epoch 17/20
563/563 [==============================] - 1s 2ms/step - loss: 4.6780e-04 - acc: 1.0000
Epoch 18/20
563/563 [==============================] - 1s 2ms/step - loss: 4.1457e-04 - acc: 1.0000
Epoch 19/20
563/563 [==============================] - 1s 2ms/step - loss: 3.6042e-04 - acc: 1.0000
Epoch 20/20
187/187 [==============================] - 0s 2ms/step
Epoch 1/20
562/562 [==============================] - 3s 5ms/step - loss: 0.6903 - acc: 0.5623
Epoch 2/20
562/562 [==============================] - 1s 2ms/step - loss: 0.6329 - acc: 0.8185
Epoch 3/20
562/562 [==============================] - 2s 3ms/step - loss: 0.4046 - acc: 0.9039
Epoch 4/20
562/562 [==============================] - 2s 3ms/step - loss: 0.1508 - acc: 0.9733
Epoch 5/20
562/562 [==============================] - 2s 4ms/step - loss: 0.0498 - acc: 0.9947
Epoch 6/20
562/562 [==============================] - 2s 3ms/step - lo

...

563/563 [==============================] - 1s 3ms/step - loss: 4.6390e-04 - acc: 1.0000
Epoch 20/20
187/187 [==============================] - 1s 4ms/step
Epoch 1/20
562/562 [==============================] - 5s 9ms/step - loss: 0.6840 - acc: 0.5730
Epoch 2/20
562/562 [==============================] - 2s 3ms/step - loss: 0.5553 - acc: 0.8808
Epoch 3/20
562/562 [==============================] - 2s 3ms/step - loss: 0.2916 - acc: 0.9342
Epoch 4/20
562/562 [==============================] - 2s 3ms/step - loss: 0.0835 - acc: 0.9875
Epoch 5/20
562/562 [==============================] - 2s 3ms/step - loss: 0.0271 - acc: 0.9982
Epoch 6/20
562/562 [==============================] - 2s 3ms/step - loss: 0.0091 - acc: 1.0000
Epoch 7/20
562/562 [==============================] - 2s 3ms/step - loss: 0.0045 - acc: 1.0000
Epoch 8/20
562/562 [==============================] - 2s 3ms/step - loss: 0.0030 - acc: 1.0000
Epoch 9/20
562/562 [==============================] - 2s 3ms/step - loss: 0.0021 - ac

187/187 [==============================] - 1s 4ms/step


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 14.0min finished


Epoch 1/20
750/750 [==============================] - 6s 8ms/step - loss: 0.6766 - acc: 0.6160
Epoch 2/20
750/750 [==============================] - 2s 3ms/step - loss: 0.5103 - acc: 0.8933
Epoch 3/20
750/750 [==============================] - 3s 3ms/step - loss: 0.2333 - acc: 0.9667
Epoch 4/20
750/750 [==============================] - 2s 3ms/step - loss: 0.0685 - acc: 0.9960
Epoch 5/20
750/750 [==============================] - 3s 4ms/step - loss: 0.0225 - acc: 0.9973
Epoch 6/20
750/750 [==============================] - 3s 4ms/step - loss: 0.0094 - acc: 1.0000
Epoch 7/20
750/750 [==============================] - 2s 3ms/step - loss: 0.0053 - acc: 1.0000
Epoch 8/20
750/750 [==============================] - 2s 3ms/step - loss: 0.0035 - acc: 1.0000
Epoch 9/20
750/750 [==============================] - 2s 3ms/step - loss: 0.0025 - acc: 1.0000
Epoch 10/20
750/750 [==============================] - 2s 3ms/step - loss: 0.0019 - acc: 1.0000
Epoch 11/20
750/750 [============================

SyntaxError: 'return' outside function (<ipython-input-29-cbbbe8ee8302>, line 14)

In [30]:
with open(output_file, 'a') as f:
    s = ('Running {} data set\nBest Accuracy : '
         '{:.4f}\n{}\nTest Accuracy : {:.4f}\n\n')
    output_string = s.format(
        source,
        grid_result.best_score_,
        grid_result.best_params_,
        test_accuracy)
    print(output_string)
    f.write(output_string)

Running imdb data set
Best Accuracy : 0.8160
{'vocab_size': 5000, 'num_filters': 32, 'maxlen': 100, 'kernel_size': 3, 'embedding_dim': 100}
Test Accuracy : 0.8120




In [44]:
# we used different kernel size, different embeddings, and different filters and found our best combination using a 
# 3 fold validaiton
new_model = grid_result.best_estimator_.model
for source in df['source'].unique():
    
    df_source = df[df['source'] == source]
    sentences = df_source['sentence'].values
    y = df_source['label'].values
    
    #preprocess X in the same way we did for our model
    X = tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(X,padding='post',maxlen=maxlen)
    
    loss, accuracy = new_model.evaluate(X, y, verbose=False)
    
    print('Accuracy for ',source,' data: ', accuracy)




Accuracy for  yelp  data:  0.953
Accuracy for  amazon  data:  0.728
Accuracy for  imdb  data:  0.7152406413924886


In [45]:
# quite the bummer that we couldnt do much better. It does seem that we are limited by our dataset